<a href="https://colab.research.google.com/github/Bryan-Az/Advanced-Prompt-Engineering/blob/main/notebooks/Multiple_Thought_Techniques.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chain of Thought (CoT) and Tree of Thought (ToT)

I attempted to use the open-source pip package 'tree-of-thoughts' with swarm capability within both colab and locally and both times it failed to build using the instructions provided by the developers. Therefore I am only looking at the simpler Chain of Thought method as well as the simpler ToT 'decorator' implementation using only langchain.

## Imports and Installs

In [ ]:
#!pip3 install -U swarms pydantic swarm-models==0.2.2 \
#"tensorflow>=2.17.0,<2.18.0" \
#"tensorflow_metadata>=1.13.0,<2.0.0" \
#"langchain-core>=0.1.33,<0.2.0" \
#"langchain" \
#"langchain-community==0.0.29" \
#"langchain-text-splitters" \
#"protobuf>=4.21.0,<5.0.0" \
#tree-of-thoughts

In [4]:
%%capture
!pip install openai langchain-community

In [6]:
import os
import openai
from langchain import PromptTemplate, LLMChain
from langchain.llms import OpenAI
from google.colab import userdata
import json

In [7]:
openai_api_key = userdata.get('openai')

## Loading Sample CoT & ToT Prompt Templates

In [29]:
prompt = 'What are the common symptoms of COVID-19? Think through this critically and note logical steps leading up to your answer.'

In [41]:
tot_meta_prompt = 'Imagine three difference experts are answering this question. All experts will write down each step of their thinking and share it with the group. They will each critique their response, and all the responses of the others. They will check their answers based on healthcare science. They will go through steps until they reach a conclusion. If at any time there is a flaw in their logic they will backtrack to where the flaw occured in their logical reasoning. The question is {question}. The experts reasoning is ...'

In [43]:
tot_fuser_prompt = '''Several experts have been asked this question: {question} And their answers are {answer} Which answer is the most likely? The most likely answer is...'''

## Chain of Thought

In [ ]:
# this model will be used on its own to demonstrate a single chain of thought
llm1 = OpenAI(temperature=0.7, max_tokens=3000, openai_api_key=openai_api_key)

In [31]:
response_CoT = llm1.generate([prompt])

In the response using the Chain-of-Thought method, the model was able to answer by splitting up its' response into bulleted step-by-step responses.

In [64]:
response_CoT.generations[0][0].text.replace('\n', '')

"1. Understanding COVID-19: COVID-19 is a highly infectious respiratory illness caused by the novel coronavirus SARS-CoV-2. It was first identified in Wuhan, China in 2019 and has since spread globally, resulting in a pandemic.2. How is COVID-19 transmitted: The virus is primarily spread through respiratory droplets when an infected person talks, coughs, or sneezes. It can also be transmitted by coming into contact with contaminated surfaces and then touching one's face.3. Incubation period: The incubation period for COVID-19 is typically 2-14 days, with an average of 5-6 days. This means that symptoms may not appear immediately after exposure to the virus.4. Common symptoms: The most common symptoms of COVID-19 include fever, dry cough, and fatigue. These symptoms are similar to the flu or a common cold, making it difficult to distinguish between them without a medical test.5. Other symptoms: In addition to the three main symptoms, COVID-19 can also cause shortness of breath, body ach

## 'Decorated' Tree of Thought

### Initializing more Models as the 'Tree Strategy'

In [8]:
llm2 = OpenAI(temperature=0.7, max_tokens=3000, openai_api_key=openai_api_key)
llm3 = OpenAI(temperature=0.7, max_tokens=3000, openai_api_key=openai_api_key)

<ipython-input-8-a06b0c6fe2d0>:1: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  llm1 = OpenAI(temperature=0.7, max_tokens=3000, openai_api_key=openai_api_key)


In [44]:
prompt1 = PromptTemplate(template=tot_meta_prompt, input_variables=['question'])
prompt2 = PromptTemplate(template=tot_meta_prompt, input_variables=['question'])
prompt3 = PromptTemplate(template=tot_meta_prompt, input_variables=['question'])

In [45]:
llm_chain_1 = LLMChain(prompt=prompt1, llm=llm1)
llm_chain_2 = LLMChain(prompt=prompt2, llm=llm2)
llm_chain_3 = LLMChain(prompt=prompt3, llm=llm3)

<ipython-input-45-7dbc3eaa523c>:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  llm_chain_1 = LLMChain(prompt=prompt1, llm=llm1)


In [46]:
response_1 = llm_chain_1.run(prompt)
response_2 = llm_chain_2.run(prompt)
response_3 = llm_chain_3.run(prompt)

<ipython-input-46-6128815bec77>:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response_1 = llm_chain_1.run(prompt)


In [55]:
fuser_model = OpenAI(temperature=0.1, max_tokens=1500, openai_api_key=openai_api_key)

In [56]:
answer = response_1 + response_2 + response_3
fusion = PromptTemplate(template=tot_fuser_prompt, input_variables=['question', 'answer'])
fusion_chain = LLMChain(prompt=fusion, llm=fuser_model)
output = fusion_chain.run(question=prompt, answer=answer)

In [58]:
display(f"The fuser models response was: {output}")

'The fuser models response was:  that the common symptoms of COVID-19 are fever, cough, and shortness of breath, with other symptoms including fatigue, body aches, headache, loss of taste or smell, sore throat, and congestion or runny nose. These symptoms may vary in severity and can also overlap with symptoms of other respiratory illnesses.'